# Géo-codage des éditeurs

In [3]:
import requests
from urllib.parse import urlencode, quote_plus
import pandas as pd
import json

## Test sur échantillon

In [4]:
editeurs = pd.read_csv('editeurs.csv')
editeurs.Editeur

0                                         Elsevier Masson
1                                   LE JOURNAL PAROISSIAL
2                                              LEXISNEXIS
3                                  BAYARD SERVICE EDITION
4                                   EDITIONS LEGISLATIVES
5                                             PUBLIHEBDOS
6                                           BAYARD PRESSE
7                                         EDITIONS DALLOZ
8                                   WOLTERS KLUWER FRANCE
9                                      EDITIONS LARIVIERE
10                                                 ORACOM
11                                           PRISMA MEDIA
12                                           MILAN PRESSE
13                                       DIVERTI EDITIONS
14                  SOCIETE DES EDITIONS FRANCIS LEFEBVRE
15                             MONDADORI MAGAZINES FRANCE
16                                         FLEURUS PRESSE
17            

In [10]:
url = "https://entreprise.data.gouv.fr/api/sirene/v1/full_text/{name}"

def check(name):
    results = []
    res = requests.get(url.format(name=name)).json()
#     siren = { etablissement['siren'] for etablissement in res['etablissement'] }
#     result['siren'] = siren
#     result['points'] = list({ (etablissement['latitude'], etablissement['longitude']) for etablissement in res['etablissement'] })

    results = res['etablissement']

    for r in results:
        r["source"] = name
    
    return results

etablissements = []

for e in list(editeurs.Editeur)[0:10]:
    etablissements += check(e)

etablissements = pd.DataFrame(etablissements)

In [17]:
list(etablissements)

['activite_principale',
 'activite_principale_entreprise',
 'activite_principale_entreprise_esa',
 'activite_principale_registre_metier',
 'arrondissement',
 'canton',
 'caractere_auxiliaire',
 'caractere_productif',
 'caractere_productif_entreprise',
 'categorie_entreprise',
 'cedex',
 'civilite',
 'code_postal',
 'commune',
 'created_at',
 'date_creation',
 'date_creation_entreprise',
 'date_debut_activite',
 'date_ess',
 'date_evenement',
 'date_introduction_base_diffusion',
 'date_introduction_base_diffusion_entreprise',
 'date_mise_a_jour',
 'date_reactivation_entreprise',
 'date_reactivation_etablissement',
 'date_validite_activite_principale',
 'date_validite_activite_principale_entreprise',
 'date_validite_effectif_salarie',
 'date_validite_effectif_salarie_entreprise',
 'date_validite_rubrique_niveau_entreprise_esa',
 'departement',
 'departement_commune_siege',
 'departement_unite_urbaine',
 'deuxieme_activite_secondaire_entreprise_esa',
 'email',
 'enseigne',
 'etablissement

In [19]:
etablissements[ ["source", "nom_raison_sociale", "siren", "activite_principale"] ]

,source,nom_raison_sociale,siren,activite_principale
0,Elsevier Masson,ELSEVIER MASSON SAS,542037031,5811Z
1,LE JOURNAL PAROISSIAL,LE JOURNAL PAROISSIAL,423670033,9499Z
2,LE JOURNAL PAROISSIAL,LE JOURNAL PAROISSIAL,489146498,5814Z
3,LE JOURNAL PAROISSIAL,JOURNAL PAROISSIAL LE LIEN,779574938,5814Z
4,LE JOURNAL PAROISSIAL,JOURNAL PAROISSIAL PERNES FONT,783242571,1811Z
5,LE JOURNAL PAROISSIAL,ASS JOURNAL PAROISSIAL QUATRE VENTS,399987494,9499Z
6,LEXISNEXIS,LEXISNEXIS,415364686,9420Z
7,LEXISNEXIS,LEXISNEXIS SA,552029431,5829C
8,LEXISNEXIS,LEXISNEXIS SA,552029431,5814Z
9,LEXISNEXIS,LEXISNEXIS SA,552029431,5814Z


In [6]:
import IPython


In [11]:
geojson = {'type':'FeatureCollection', 'features':[]}

for etablissement in etablissements:
    feature = {
        'type': 'Feature',
        'properties': {},
        'geometry': {'type':'Point', 'coordinates':[]}
    }
    
    feature['geometry']['coordinates'] = [ etablissement['points'][0][1], etablissement['points'][0][0] ]
    
    geojson['features'].append(feature)

print(geojson)
    
IPython.display.display({'application/geo+json': geojson}, raw=True)

{'features': [{'type': 'Feature', 'geometry': {'coordinates': ['2.260111', '48.826712'], 'type': 'Point'}, 'properties': {}}, {'type': 'Feature', 'geometry': {'coordinates': ['0.558783', '46.041517'], 'type': 'Point'}, 'properties': {}}, {'type': 'Feature', 'geometry': {'coordinates': ['1.844011', '47.947301'], 'type': 'Point'}, 'properties': {}}, {'type': 'Feature', 'geometry': {'coordinates': ['3.059165', '50.687935'], 'type': 'Point'}, 'properties': {}}, {'type': 'Feature', 'geometry': {'coordinates': ['2.303984', '48.812754'], 'type': 'Point'}, 'properties': {}}, {'type': 'Feature', 'geometry': {'coordinates': ['-3.047081', '48.778602'], 'type': 'Point'}, 'properties': {}}, {'type': 'Feature', 'geometry': {'coordinates': ['5.368183', '43.287336'], 'type': 'Point'}, 'properties': {}}, {'type': 'Feature', 'geometry': {'coordinates': ['4.851131', '45.768641'], 'type': 'Point'}, 'properties': {}}, {'type': 'Feature', 'geometry': {'coordinates': ['2.126629', '48.800254'], 'type': 'Point